# Formulations compactes

Dans ce notebook, nous définissons les formulations compactes écrites en 4e année, pour résoudre le problème de transplantation d'organes en domino dans 3 cas différents: 

- En considérant qu'il n'y a pas de limite de taille sur les cycles solution
- En considérant qu'on ne peut réaliser que des associations 2 à 2 (cycles de taille 2)
- En considérant une taille maximale K, et en ajoutant des hôpitaux "virtuels" dans lesquels les transplantations sont faites

## Fonction déterminant les cycles solutions

La méthode suivante récupère la variable réponse des modèles (1) et (2), et renvoie les cycles solutions de ces deux formulations.

In [ ]:
function compute_solution(model)
    """
    Crée les cycles solutions rendu par le modèle fourni.
    
    Entrées : 
        - model : JuMP Model - Modèle résolu du problème
    
    Sorties : 
        - cycles : Array - Cycles solutions
    """
    # Récupération des valeurs optimales des variables et de la valeur optimale
    X = value.(model[:x])

    # Récupération des arcs solution
    arcs = []
    for (i,j) in axes(X)[1]
        if abs(X[(i,j)]-1) < 1e-6
            push!(arcs,(i,j))
        end
    end
    
    
    # Création de la liste des cycles solution
    cycles = []
    test = true

    cycle = [] 
    next = arcs[1]
    
    # Premier affichage
    println("Cycles : ")

    # Tant qu'il reste des arcs à traiter
    while test
        # On ajoute l'arc au cycle courant
        push!(cycle,next)

        # Et on le supprime de la liste des arcs à traiter
        arcs = filter(t -> t != next,arcs)

        # On met à jour l'arc suivant
        next = filter(t -> next[2] in t, arcs)
        if length(next) > 0
            # Cas où le cycle courant n'est pas terminé
            next = next[1]
        else
            # Cas où le cycle courant est terminé
            ## On l'ajoute au vecteur des cycles solution
            push!(cycles,cycle)
            print("  - ")
            println(cycle)

            if length(arcs) > 0
                # S'il reste des arcs à parcourir, on recommence
                next = arcs[1]
                cycle = []
            else 
                # Sinon, on sort de la boucle
                test = false
            end
        end
    end 
    return cycles
end

## Formulation sans limite de taille

In [ ]:
function compact_cycle_infini(graph,logs)
    """
    Crée le modèle linéaire associé au problème de transplantation en domino, sans contrainte de longueur.

    Entrées : 
        - graph : Array   - Vecteur contenant les propriétés du graphe (n,A,W,preds,succ)
        - logs  : Boolean - Booléen pour l'affichage de la solution

    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Récupération du graphe d'étude
    n = graph[1]
    A = graph[2]
    W = graph[3]
    preds = graph[4]
    succ = graph[5]
    
    # Création du vecteur des sommets
    V=Array(1:n)
    
    # Création du modèle JuMP
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)
    
    # Variables
    ## pour tout arc (i,j), x[(i,j)] = 1 si l'arc (i,j) est choisi dans la solution, 0 sinon
    @variable(model, x[A], Bin);
    
    # Fonction Objectif: On souhaite maximiser le coût des transplantations
    @objective(model, Max, sum(x[a]*W[a[1],a[2]] for a in A));
    
    # Contraintes
    ## Un couple peut donner, au plus, à un seul autre couple
    @constraint(model,c1[i in V],sum(x[(i,j)] for j in succ[i])<=1)
    
    ## Si un couple donne à un autre, alors il doit recevoir
    @constraint(model,c2[i in V],sum(x[(i,j)] for j in succ[i])==sum(x[(j,i)] for j in preds[i]))
    
    # Résolution du modèle
    timer = @timed optimize!(model);  
    
    # Affichage de la solution si souhaité
    if logs
        println("***** Taille Infinie *****\n");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
    return objective_value(model),model,timer;
end

## Formulation avec cycles de taille $\le$ 2

In [ ]:
function compact_cycle_2(graph,logs)
    """
    Crée le modèle de transplantation avec cycles de taille 2

    Entrées : 
        - graph : Array   - Vecteur contenant les propriétés du graphe (n,A,W)
        - logs  : Boolean - Booléen pour l'affichage de la solution

    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Récupération des propriétés du graphe
    n = graph[1]
    A = graph[2]
    W = graph[3]
    
    # Vecteur des noeuds
    V=Array(1:n)
    
    # On crée les listes A_2 et V_2
    A_2=[]
    V_2=[]
    for a in A
        b=(a[2],a[1])
        if b in A 
            push!(A_2,b)
            if !(a[1] in V_2)
                push!(V_2,a[1])
            end
            if !(a[2] in V_2)
                push!(V_2,a[2])
            end
        end
    end
    
    # Liste des successeurs de chaque sommet de vertices
    succ=[]
    for i in V
        push!(succ,[])
    end 
    
    for a in A_2
        push!(succ[a[1]],a[2])
    end
    
    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)
    
    # Variables
    ## pour tout arc (i,j) de A_2, x[i,j]=1 si l'arc est choisi dans la solution, 0 sinon
    @variable(model, x[A_2], Bin);
    
    # Fonction objectif: On souhaite maximiser la somme des coûts des transplantations
    @objective(model, Max, sum(x[a]*W[a[1],a[2]] for a in A_2));
    
    # Contraintes 
    ## Un couple peut donner, au plus, 1 fois
    @constraint(model,c1[i in V_2],sum(x[(i,j)] for j in succ[i])<=1)
    ## Si un couple donne à un autre couple, alors il doit aussi recevoir
    @constraint(model,c2[(i,j) in A_2],x[(i,j)]==x[(j,i)])
    
    # Résolution du modèle
    timer = @timed optimize!(model);  
    
    # Affichage de la solution si souhaité
    if logs
        println("***** Taille 2 *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
    
    return objective_value(model), model,timer;
end

## Formulation par hôpital

Pour le modèle par hôpital, on commence par définir une fonction qui construit et affiche les cycles solution de cette formulation.

In [2]:
function compute_solution_K(model,n)
    """
    Fonction qui construit et affiche les cycles solutions de la formulation par hôpital.
    
    Entrées : 
        - model : JuMP Model - Modèle résolu du problème
        - n     : Integer    - Nombre de sommets dans le graphe
    """

    # Récupération des valeurs optimales des variables et de la valeur optimale
    X = value.(model[:x])
    
    # Récupération des arcs solution
    arcs = [[] for i in 1:n]
    for (i,j,k) in axes(X)[1]
        if abs(X[(i,j,k)]-1) < 1e-6
            push!(arcs[k],(i,j))
        end
    end
    
    cycles = filter(x -> !isempty(x), arcs)

    println("Cycles :")
    # Tant qu'il reste des arcs à traiter
    for c in range(1,length(cycles))
        arcs = cycles[c]
        next = arcs[1]
        cycle = []
        
        while(length(arcs) > 0)
            # On ajoute l'arc au cycle courant
            push!(cycle,next)
            # On retire l'arc ajouté de la liste "arcs"
            arcs = filter(t -> t != next,arcs)

            # On met à jour l'arc suivant
            next = filter(t -> next[2] in t, arcs)
            
            if length(next) > 0
                # Cas où le cycle courant n'est pas terminé
                next = next[1]
            else
                # Cas où le cycle courant est terminé
                ## On l'ajoute au vecteur des cycles solution
                push!(cycles,cycle)
                print("  - ")
                println(cycle)
            end
        end
    end 
end

compute_solution_K (generic function with 1 method)

In [ ]:
function compact_cycle_K(graph,K,logs)
    """
    Crée le modèle de transplantations en dominos par hôpital.

    Entrées : 
        - graph : Array   - Vecteur contenant les propriétés du graphe (n,A,W,preds,succ)
        - K     : Integer - Taille maximale des cycles de transplantation
        - logs  : Boolean - Booléen pour l'affichage de la solution
    
    Sortie : 
        - objective_value(model) : Float      - Valeur optimale du modèle résolu 
        - model                  : JuMP Model - Modèle du problème
        - timer                  : Float      - Temps de calcul
    """
    # Récupération des propriétés du graphe
    n = graph[1]
    A = graph[2]
    W = graph[3]
    preds = graph[4]
    succ = graph[5]
    
    # Vecteur des sommets et vecteur des hôpitaux
    nb_max = Integer(floor(n/2))
    H = Array(1:nb_max)
    V = Array(1:n)
    
    # Variables
    ## On crée un ensemble tri-dimensionnel (i,j,k), pour tout arc (i,j) et hôpital k
    triplets=[]
    for a in A
        for k in H
            push!(triplets,(a[1],a[2],k))
        end
    end 
        
    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)

    # Variables
    ## x[i,j,k]=1 la transplantation i->j est faite dans l'hôpital k
    @variable(model, x[triplets], Bin);
    
    # Fonction objectif: On souhaite maximiser la somme des coûts des transplantations
    @objective(model, Max, sum(x[a]*W[a[1],a[2]] for a in triplets));
    
    # Contraintes
    ## Un couple peut donner, au plus, à un autre couple
    @constraint(model,c1[i in V],sum(x[(i,j,k)] for j in succ[i] for k in H)<=1)
    ## Si un couple donne dans un hôpital k, alors il doit recevoir dans ce même hôpital
    @constraint(model,c2[i in V, k in H],sum(x[(i,j,k)] for j in succ[i])==sum(x[(j,i,k)] for j in preds[i]))
    ## Un hôpital peut faire, au plus, K transplantations simultanées
    @constraint(model,c3[k in H],sum(x[(i,j,k)] for (i,j) in A)<=K)
        
    
    timer = @timed optimize!(model);  
    # Affichage de la solution si souhaité
    if logs 
        println("***** Hôpital *****");
        # Statistiques de la solution
        println("Statut de la solution = ", termination_status(model));
        println("Valeur optimale = ", objective_value(model));
        println("Temps = ", timer[2])
    end
            
    return objective_value(model),model,timer;
end

In [1]:
println("Formulations compactes importées avec succès.")

Fonctions importées avec succès
